# <u>Explanatory Logistic Regression</u> - cleaned and runnning

In [ ]:
# Importing Python packages and modules
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import folium
import statsmodels.formula.api as smf
import warnings
from folium.plugins import MarkerCluster
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.calibration import calibration_curve
from shapely.geometry import Point
from branca.colormap import linear
from patsy import dmatrix
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from IPython.display import display

In [ ]:
# Importing log transformed and scaled buffer count (see 2.T_test notebook) 
df = gpd.read_file("df_log.csv")

In [ ]:
df.sample(5)

------------------------------------------

# Cleaning and examing the data

#### Creating a GeoDataFrame

In [ ]:
# Making sure the Latitude and Longitude are numeric
df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")

# Creating a Point geometry column
df["Point geometry"] = df.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)

# Making the GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry="Point geometry", crs="EPSG:4326")

# Preview
gdf.head()

In [ ]:
# Creating a new binary is_strandings column
gdf['is_stranding'] = gdf['Data'].apply(lambda x: 1 if x == 'Strandings points' else 0)

# Ensuring 'Buffer size' is treated as integer
if gdf['Buffer size'].dtype == 'object':
    gdf['Buffer size'] = gdf['Buffer size'].str.replace('m', '').astype(int)

In [ ]:
#Checking crs
print("CRS:", gdf.crs)

In [ ]:
#Checking for null values across all buffers
gdf.info()

----------------------------------

## Explanatory Logistic Regression 

#### Preliminary t-tests indicated stronger performance with smaller buffer sizes, whereas predictive logistic regression models achieved higher accuracy with larger buffers. To refine the choice of buffer size, I next applied explanatory logistic regression to better evaluate the underlying relationships before selecting an appropriate scale for further analysis.

In [ ]:
# Setting what data to use, creating a copy of the gdf to work with
def run_logit_by_buffers(
    gdf,
    buffers=(500, 1000, 1500, 3000, 5000),
    predictors=('Building_trans', 'Road_trans', 'Bathymetry_trans', 'Other_points_trans'),
    buffer_col='Buffer size',
    data_col='Data',
    target_label='Strandings points'):
    df = gdf.copy()

     # Making sure predictors are numeric
    for col in predictors:
         df[col] = pd.to_numeric(df[col], errors='coerce')

    # Dropping any null values for predictors/target
    df = df.dropna(subset=list(predictors) + ['is_stranding'])

    # Creating containers
    model_rows = []
    or_rows = []
    vif_rows = []
    results = {}
    # looping over the buffers and fitting the logit model per buffer
    for b in buffers:
        subset = df[df[buffer_col] == b].copy()
        if subset.empty:
            model_rows.append({'Buffer': b, 'n': 0, 'AIC': np.nan, 'Pseudo_R2': np.nan})
            continue
        # bulding X from the predictors and intercept, setting y as 'is_stranding'
        X = subset[list(predictors)]
        X = sm.add_constant(X, has_constant='add')
        y = subset['is_stranding']

        # Skipping an drecord any NaNs to prevent degenerate MLE
        if X.shape[0] <= X.shape[1]:
            model_rows.append({'Buffer': b, 'n': len(subset), 'AIC': np.nan, 'Pseudo_R2': np.nan})
            continue
        # Storing results 
        try:
            model = sm.Logit(y, X)
            res = model.fit(disp=False, maxiter=100)
        except Exception as e:
            # model could fail with complete separation, etc.
            model_rows.append({'Buffer': b, 'n': len(subset), 'AIC': np.nan, 'Pseudo_R2': np.nan, 'error': str(e)})
            continue
        
        results[b] = res

        # Collecting per-buffer model stats
        model_rows.append({
            'Buffer': b,
            'n': int(len(subset)),
            'AIC': float(res.aic),
            'Pseudo_R2': float(res.prsquared),
            'LL': float(res.llf)})

        # Computing odds ratios (exp(coef)) with 95% CI and p-values
        conf = res.conf_int()
        conf.columns = ['CI_low', 'CI_high']
        coef = res.params.rename('coef')
        pvals = res.pvalues.rename('pval')
        or_df = pd.concat([coef, conf, pvals], axis=1).reset_index().rename(columns={'index':'Variable'})
        or_df['Buffer'] = b
        or_df['OR'] = np.exp(or_df['coef'])
        or_df['CI_low'] = np.exp(or_df['CI_low'])
        or_df['CI_high'] = np.exp(or_df['CI_high'])
        or_rows.append(or_df[['Buffer','Variable','OR','CI_low','CI_high','pval']])

        # computing VIF for multicollinearity
        vif = pd.DataFrame({
            'Buffer': b,
            'Feature': X.columns,
            'VIF': [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]})
        vif_rows.append(vif)
    #Returning tidy tables and raw models
    summary_df = pd.DataFrame(model_rows).sort_values('Buffer').reset_index(drop=True)
    or_table = pd.concat(or_rows, ignore_index=True) if or_rows else pd.DataFrame()
    vif_table = pd.concat(vif_rows, ignore_index=True) if vif_rows else pd.DataFrame()

    return summary_df, or_table, vif_table, results

summary, ORs, VIFs, models = run_logit_by_buffers(gdf)
# Prings per-buffer n, AIC, pseudo-R2
print(summary)  
# odds ratios with 95% CI & p-values
print(ORs.head(25))
# multicollinearity diagnostics
print(VIFs.head(25)) 

# Example: see full statsmodels summary for which a single buffer size, to take a more indepth look
print(models[500].summary())

#### Key points: smaller buffers (500m - 1000m) produce better fitting, more explanatory models with highest pseudo-R² (500m ~0.25) and lower AIC (500m ~52,417). Roads and buildings are the strongest positive predictors of strandings, other points are influential and multicollinearity isn't a concern. 

#### Deeper dive on 500m buffer shows: Strandings are most strongly driven by spatial clustering and human presence, while bathymetry reduces risk.

In [ ]:
# Save all outputs to one text file for your appendix
with open("Method_results_images/Logistic_Regression_summary.txt", "w") as f:
    # Write summary table
    f.write("=== Summary ===\n")
    f.write(summary.to_string(index=False))
    f.write("\n\n")

with open("Method_results_images/Logistic_Regression_Odds_Ratio.txt", "w") as f:    
    f.write("=== Odds Ratios ===\n")
    f.write(ORs.to_string(index=False))
    f.write("\n\n")

with open("Method_results_images/Logistic_Regression_VIF_Table.txt", "w") as f:    
    f.write("=== VIF Table ===\n")
    f.write(VIFs.to_string(index=False))
    f.write("\n\n")

with open("Method_results_images/Logistic_Regression_Full_Model_Summary.txt", "w") as f:    
    f.write("=== Full Model Summary (1000 m) ===\n")
    f.write(models[1000].summary().as_text())

print("Saved everything to method results images.txt")




## Visulising the results

#### Model fit summary

In [ ]:
#Removing warnings for aesthetics
warnings.filterwarnings("ignore", category=FutureWarning)
# Plotting, displaying and visulising AIC and Pseudo-R graphs
fig, ax = plt.subplots(1, 2, figsize=(12,5))
sns.lineplot(data=summary, x="Buffer", y="AIC", marker="o", ax=ax[0])
ax[0].set_title("AIC vs Buffer Size")
sns.lineplot(data=summary, x="Buffer", y="Pseudo_R2", marker="o", ax=ax[1])
ax[1].set_title("Pseudo-R² vs Buffer Size")
plt.tight_layout()
plt.tight_layout()
plt.savefig("Method_results_images/Logistic_Regression_3_AIC_Pseudo_R_vs_buffer_size.png", dpi=150, bbox_inches="tight")
plt.show()


#### AIC vs buffer size: increases as buffer size increases, lower AIC = better fit so 500m buffer performs best, while larger buffers perform worse. 
#### Pseudo-R²: decreases as buffersize increases, 500m buffer performs best again, explaining the most variation in strandings presence


#### Odds ratio

In [ ]:
# Tidying up the 
name_map = {
    'Building_trans':'Buildings',
    'Road_trans':'Roads',
    'Bathymetry_trans':'Bathymetry',
    'Other_points_trans':'Other strandings'}

ORs['Predictor'] = ORs['Variable'].map(name_map).fillna(ORs['Variable'])

# Make sure Buffer is numeric & ordered
ORs['Buffer'] = pd.to_numeric(ORs['Buffer'], errors='coerce')
buffer_order = [500, 1000, 1500, 3000, 5000]
ORs['Buffer'] = pd.Categorical(ORs['Buffer'], categories=buffer_order, ordered=True)

# Custom palette
palette = {
    'Buildings': '#1f77b4',
    'Roads': '#ff7f0e',
    'Bathymetry': '#2ca02c',
    'Other strandings': '#d62728'}

# Plot
plt.figure(figsize=(9,6))
sns.lineplot(data=ORs[ORs['Variable']!='const'],
             x='Buffer', y='OR', hue='Predictor',
             palette=palette, marker='o')
plt.yscale('log')
plt.axhline(1, color='gray', ls='--', lw=1.5)
plt.xlabel('Buffer size')
plt.ylabel('Odds ratio (log scale)')
plt.title('Odds ratios across buffers')
plt.tight_layout()
plt.savefig("Method_results_images/Logistic_Regression_Odds_Ratios_buffer_size.png", dpi=150, bbox_inches="tight")
plt.show()




#### Odds ratio: local factors, bathymetry and other strandings dominate at smaller scales, human presence (roads, buildings) remain consistently important wiht roads increasing in finfluence at larger scales. The strongest predictor is presence of other strandings. 

#### VIFs

In [ ]:
# Cleaning up the legend names
name_map = {
    "Building_trans": "Buildings",
    "Road_trans": "Roads",
    "Bathymetry_trans": "Bathymetry",
    "Other_points_trans": "Other strandings"}

# Appling mapping to new column
VIFs['Predictor'] = VIFs['Feature'].map(name_map).fillna(VIFs['Feature'])

# Plot
plt.figure(figsize=(10,6))
sns.lineplot(data=VIFs[VIFs["Predictor"]!="const"], 
             x="Buffer", y="VIF", hue="Predictor", marker="o")

plt.axhline(5, ls="--", c="red", label="VIF=5 (multicollinearity threshold)")
plt.title("VIF by Buffer Size")
plt.legend(title="Predictor")
plt.savefig("Method_results_images/Logistic_Regression_VIF_buffer_size.png", dpi=150, bbox_inches="tight")
plt.show()


#### VIF: Multicollinearity is not a concern in the models, roads and buildings show (expected) correlation, especially as the buffers get bigger.

------------------

# Residual analysis

#### Small (500m buffers) vs large (3000m buffer)

In [ ]:
def residual_diagnostics(res):
    """Convenience: return fitted, pearson residuals, leverage, Cook's D."""
    infl = res.get_influence()
    return (res.fittedvalues,
            res.resid_pearson,
            infl.hat_matrix_diag,
            infl.cooks_distance[0])

def plot_residuals_clean(res, buffer_size, label_mode="none", top_k=10, quantile=0.99):
    """
    Residuals vs Fitted + Leverage vs Residuals for a statsmodels Logit result.
    label_mode: "none" (default), "topk", or "quantile"
    """
    fitted, pearson, leverage, cooks_d = residual_diagnostics(res)

    # Residuals vs Fitted 
    plt.figure(figsize=(6,4))
    plt.scatter(fitted, pearson, s=8, alpha=0.5)
    plt.axhline(0, color='red', ls='--', lw=1)
    plt.xlabel("Fitted values"); plt.ylabel("Pearson residuals")
    plt.title(f"Residuals vs Fitted ({buffer_size} m)")
    plt.tight_layout(); plt.show()

    # Leverage vs Residuals
    plt.figure(figsize=(6,4))
    plt.scatter(leverage, pearson, s=8, alpha=0.5)
    plt.axhline(0, color='red', ls='--', lw=1)
    plt.xlabel("Leverage"); plt.ylabel("Pearson residuals")
    plt.title(f"Leverage vs Residuals ({buffer_size} m)")

    if label_mode == "topk":
        idx = np.argsort(cooks_d)[-top_k:]
    elif label_mode == "quantile":
        thr = np.quantile(cooks_d, quantile)
        idx = np.where(cooks_d >= thr)[0]
    else:
        idx = []

    # Only annotate a *few* influential points 
    for i in idx:
        plt.annotate(str(int(i)), (leverage[i], pearson[i]),
                     xytext=(2,2), textcoords="offset points", fontsize=8)

    plt.tight_layout(); plt.show()

def plot_residuals_highlight(res, buffer_size, quantile=0.99):
    """Highlight (don’t label) the most influential observations by Cook’s D."""
    _, pearson, leverage, cooks_d = residual_diagnostics(res)
    thr = np.quantile(cooks_d, quantile)
    hi = cooks_d >= thr

    plt.figure(figsize=(6,4))
    plt.scatter(leverage[~hi], pearson[~hi], s=8, alpha=0.35, label="Others")
    plt.scatter(leverage[hi], pearson[hi], s=24, alpha=0.9, edgecolor='k',
                label=f"Top {int((1-quantile)*100)}% Cook's D")
    plt.axhline(0, color='red', ls='--', lw=1)
    plt.xlabel("Leverage"); plt.ylabel("Pearson residuals")
    plt.title(f"Leverage vs Residuals ({buffer_size} m)")
    plt.legend(frameon=False)
    plt.tight_layout(); 
    plt.savefig("Method_results_images/Logistic_Regression_500m_Pearson_residuals.png", dpi=150, bbox_inches="tight")
    plt.show()


In [ ]:
plot_residuals_clean(models[500], 500, label_mode="none")
plot_residuals_clean(models[3000], 3000, label_mode="none")

#### Plots indicate that logistic regression model performes more consistently at 500m buffers size compared to the 3000m. 

#### Filters only the high deviance residuals (> 2)) for 500m buffer

In [ ]:
# Settings
BUF_M = 500                     
PREDICTORS = ['Building_trans','Road_trans','Bathymetry_trans','Other_points_trans']
THRESH = 2.0                  
USE_DEV = True                 
SAVE_HTML = True             

# Creating a subset 500
df = gdf.copy()

# normalizing buffer to numeric meters 
if df['Buffer size'].dtype == 'object':
    df['_buf_m'] = df['Buffer size'].astype(str).str.replace('m', '', regex=False).astype(int)
else:
    df['_buf_m'] = df['Buffer size']

sub = df.loc[df['_buf_m'] == BUF_M].copy()

# making sure geometry is WGS84 for mapping
if 'geometry' not in sub.columns and 'Point geometry' in sub.columns:
    sub = sub.rename(columns={'Point geometry':'geometry'})
sub = gpd.GeoDataFrame(sub, geometry='geometry', crs=gdf.crs)
sub = sub.to_crs(4326)

# labelling target and keeping usable rows
sub['is_stranding'] = (sub['Data'] == 'Strandings points').astype(int)
for c in PREDICTORS:
    sub[c] = pd.to_numeric(sub[c], errors='coerce')
sub = sub.dropna(subset=PREDICTORS + ['is_stranding'])

# Fitting logit on 500 m 
X = sm.add_constant(sub[PREDICTORS], has_constant='add')
y = sub['is_stranding'].values
res = sm.Logit(y, X).fit(disp=False, maxiter=100)

# Residuals
def binomial_deviance_residual(y, mu, eps=1e-12):
    mu = np.clip(mu, eps, 1 - eps)
    # y is 0/1
    term1 = y * np.log(np.clip((y + eps) / mu, eps, None))
    term2 = (1 - y) * np.log(np.clip((1 - y + eps) / (1 - mu), eps, None))
    return np.sign(y - mu) * np.sqrt(2.0 * (term1 + term2))

mu = res.predict(X)
if USE_DEV:
    resid = binomial_deviance_residual(y, mu)
else:
    resid = res.resid_pearson

sub['residual'] = resid
sub['_abs_resid'] = np.abs(sub['residual'])

# Filtering high residuals
hi = sub.loc[sub['_abs_resid'] > THRESH].copy()

# nothing to plot guard
if hi.empty:
    print("No points exceed the residual threshold; try lowering THRESH.")
else:
    # Map
    center = [hi.geometry.y.mean(), hi.geometry.x.mean()] if len(hi) else [55, -3]
    m = folium.Map(location=center, zoom_start=5, tiles='CartoDB positron', control_scale=True)

    vmin, vmax = float(hi['residual'].min()), float(hi['residual'].max())
    cmap = linear.RdBu_11.scale(vmin, vmax)
    cmap.caption = f"{'Deviance' if USE_DEV else 'Pearson'} residual (|res|>{THRESH})"

    def fmt(v, d=2):
        try:
            x = float(v); 
            return "NA" if pd.isna(x) else f"{x:.{d}f}"
        except Exception:
            return "NA"

    for _, r in hi.iterrows():
        popup_html = (
            f"<b>{'Deviance' if USE_DEV else 'Pearson'} residual:</b> {fmt(r['residual'])}<br>"
            + (f"<b>Building count:</b> {fmt(r.get('Building count', np.nan), 0)}<br>" if 'Building count' in r else "")
            + (f"<b>Road length:</b> {fmt(r.get('Road length', np.nan))} m" if 'Road length' in r else ""))
        
        folium.CircleMarker(
            [r.geometry.y, r.geometry.x],
            radius=5,
            color=cmap(r['residual']),
            fill=True, fill_color=cmap(r['residual']), fill_opacity=0.85,
            popup=folium.Popup(popup_html, max_width=300),
        ).add_to(m)

title_html = """
<div style="
    position: absolute;
    top: 12px;
    left: 64px;              /* nudged right from 12px -> 64px */
    z-index: 9999;
    background: white;
    padding: 6px 10px;
    border: 1px solid #777;
    border-radius: 6px;
    font-weight: 600;
    box-shadow: 0 1px 3px rgba(0,0,0,0.2);
    pointer-events: none;    /* still lets you click the zoom buttons */
">
High deviance residuals (&gt; 2) for 500m buffer
</div>
"""
m.get_root().html.add_child(folium.Element(title_html))

cmap.add_to(m)

display(m)



In [ ]:
m.save("Method_results_images/Logistic_Regression_High_Deviance_Residuals_Map_500m.html")

#### Saved as a link with Netlify [High deviance residuals (> 2)) for 500m buffer](https://high-deviance-residuals-500m.netlify.app/)

#### Filters only the high deviance residuals (> 2)) for 3000m buffer

In [ ]:
# Settings
BUF_M = 3000                    
PREDICTORS = ['Building_trans','Road_trans','Bathymetry_trans','Other_points_trans']
THRESH = 2.0                  
USE_DEV = True                 
SAVE_HTML = True             

# Creating a subset 500
df = gdf.copy()

# normalizing buffer to numeric meters 
if df['Buffer size'].dtype == 'object':
    df['_buf_m'] = df['Buffer size'].astype(str).str.replace('m', '', regex=False).astype(int)
else:
    df['_buf_m'] = df['Buffer size']

sub = df.loc[df['_buf_m'] == BUF_M].copy()

# making sure geometry is WGS84 for mapping
if 'geometry' not in sub.columns and 'Point geometry' in sub.columns:
    sub = sub.rename(columns={'Point geometry':'geometry'})
sub = gpd.GeoDataFrame(sub, geometry='geometry', crs=gdf.crs)
sub = sub.to_crs(4326)

# labelling target and keeping usable rows
sub['is_stranding'] = (sub['Data'] == 'Strandings points').astype(int)
for c in PREDICTORS:
    sub[c] = pd.to_numeric(sub[c], errors='coerce')
sub = sub.dropna(subset=PREDICTORS + ['is_stranding'])

# Fitting logit on 500 m 
X = sm.add_constant(sub[PREDICTORS], has_constant='add')
y = sub['is_stranding'].values
res = sm.Logit(y, X).fit(disp=False, maxiter=100)

# Residuals
def binomial_deviance_residual(y, mu, eps=1e-12):
    mu = np.clip(mu, eps, 1 - eps)
    # y is 0/1
    term1 = y * np.log(np.clip((y + eps) / mu, eps, None))
    term2 = (1 - y) * np.log(np.clip((1 - y + eps) / (1 - mu), eps, None))
    return np.sign(y - mu) * np.sqrt(2.0 * (term1 + term2))

mu = res.predict(X)
if USE_DEV:
    resid = binomial_deviance_residual(y, mu)
else:
    resid = res.resid_pearson

sub['residual'] = resid
sub['_abs_resid'] = np.abs(sub['residual'])

# Filtering high residuals
hi = sub.loc[sub['_abs_resid'] > THRESH].copy()

# nothing to plot guard
if hi.empty:
    print("No points exceed the residual threshold; try lowering THRESH.")
else:
    # Map
    center = [hi.geometry.y.mean(), hi.geometry.x.mean()] if len(hi) else [55, -3]
    m = folium.Map(location=center, zoom_start=5, tiles='CartoDB positron', control_scale=True)

    vmin, vmax = float(hi['residual'].min()), float(hi['residual'].max())
    cmap = linear.RdBu_11.scale(vmin, vmax)
    cmap.caption = f"{'Deviance' if USE_DEV else 'Pearson'} residual (|res|>{THRESH})"

    def fmt(v, d=2):
        try:
            x = float(v); 
            return "NA" if pd.isna(x) else f"{x:.{d}f}"
        except Exception:
            return "NA"

    for _, r in hi.iterrows():
        popup_html = (
            f"<b>{'Deviance' if USE_DEV else 'Pearson'} residual:</b> {fmt(r['residual'])}<br>"
            + (f"<b>Building count:</b> {fmt(r.get('Building count', np.nan), 0)}<br>" if 'Building count' in r else "")
            + (f"<b>Road length:</b> {fmt(r.get('Road length', np.nan))} m" if 'Road length' in r else ""))
        
        folium.CircleMarker(
            [r.geometry.y, r.geometry.x],
            radius=5,
            color=cmap(r['residual']),
            fill=True, fill_color=cmap(r['residual']), fill_opacity=0.85,
            popup=folium.Popup(popup_html, max_width=300),
        ).add_to(m)

title_html = """
<div style="
    position: absolute;
    top: 12px;
    left: 64px;              /* nudged right from 12px -> 64px */
    z-index: 9999;
    background: white;
    padding: 6px 10px;
    border: 1px solid #777;
    border-radius: 6px;
    font-weight: 600;
    box-shadow: 0 1px 3px rgba(0,0,0,0.2);
    pointer-events: none;    /* still lets you click the zoom buttons */
">
High deviance residuals (&gt; 2) for 500m buffer
</div>
"""
m.get_root().html.add_child(folium.Element(title_html))

cmap.add_to(m)

display(m)

In [ ]:
m.save("Method_results_images/Logistic_Regression_High_Deviance_Residuals_Map_3000m.html")

#### Saved as a link with Netlify [High deviance residuals (> 2)) for 3000m buffer](https://high-deviance-residuals-3000m.netlify.app/)

# Continues in 5. Under-reported Strandings Zones notebook